<a href="https://colab.research.google.com/github/mehrn79/npl_wrod2wec/blob/main/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **word2vec2 implematation**

## **preprocessing**

### **libraries for preprocessing**

In [ ]:
pip install hazm

In [20]:
from __future__ import unicode_literals
from hazm import *
from keras.preprocessing import text
from collections import Counter

### **getting the corpes & stop words**

In [4]:
with open('/content/shams.txt') as f:
  lines = f.readlines()

with open('/content/stopwords.txt') as file:
  stopLines = file.readlines()
stopWord = [item.replace('\n',"") for item in stopLines]
  

### **cleaning the corpes**

In [5]:
words= []
stemmer = Stemmer()
for sent in lines :
  words.append(word_tokenize(sent))

text_word_list=[]
corpes=[]
new_words=[]

for wordSent in words :
  cleanSent=[]
  for word in wordSent :
    if word not in stopWord :
      cleanSent.append(stemmer.stem(word))
  if(len(cleanSent)>1) :
    text_word_list+=cleanSent
    new_words.append(cleanSent)

value_of_repetition = dict(Counter(text_word_list))
repetition=[]
for word in value_of_repetition :
  if value_of_repetition.get(word)==1:
    repetition.append(word)
clear_word_list=[x for x in text_word_list if x not in repetition]

text_word_list=[]

for wordSent in new_words :
  cleanSent=[]
  for word in wordSent :
    if word not in repetition :
      cleanSent.append(word)
  if(len(cleanSent)>1) :
    text_word_list+=cleanSent
    corpes.append(' '.join(cleanSent))

### **tokenizing the corpes**

In [21]:
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(corpes)

word2id = tokenizer.word_index
id2word = {v:k for k, v in word2id.items()}


### **creating list of target words and context words**

In [ ]:
window = 2
word_lists = []

for text in corpes:
    text=word_tokenize(text)

    for i, word in enumerate(text):
        for w in range(window):
            if i + 1 + w < len(text): 
                word_lists.append([word] + [text[(i + 1 + w)]])   
            if i - w - 1 >= 0:
                word_lists.append([word] + [text[(i - w - 1)]])

### **finding unique words**

In [9]:
words_set = list(set(text_word_list))
words_set.sort()

unique_word_dict = {}
for i, word in enumerate(words_set):
  unique_word_dict.update({
    word: i
  })

## **make X & Y of train**

In [10]:
from scipy import sparse
import numpy as np
from tqdm import tqdm

n_words = len(unique_word_dict)
 
words = list(unique_word_dict.keys())

X = []
Y = []

for i, word_list in tqdm(enumerate(word_lists)):
    main_word_index = unique_word_dict.get(word_list[0])
    context_word_index = unique_word_dict.get(word_list[1])
 
    X_row = np.zeros(n_words)
    Y_row = np.zeros(n_words)

    X_row[main_word_index] = 1
    Y_row[context_word_index] = 1

    X.append(X_row)
    Y.append(Y_row)

X = np.asarray(X)
Y = np.asarray(Y)


110306it [00:05, 22042.72it/s]


## **creating training model**

In [11]:
from keras.models import Input, Model
from keras.layers import Dense

embed_size = 100

inp = Input(shape=(X.shape[1],))
x = Dense(units=embed_size, activation='linear')(inp)
x = Dense(units=Y.shape[1], activation='softmax')(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics=['categorical_accuracy'])
model.summary()
model.fit(
    x=X, 
    y=Y, 
    epochs=20,
    verbose=1
    )



Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3447)]            0         
                                                                 
 dense (Dense)               (None, 100)               344800    
                                                                 
 dense_1 (Dense)             (None, 3447)              348147    
                                                                 
Total params: 692,947
Trainable params: 692,947
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
3448/3448 [==============================] - 34s 10ms/step - loss: 7.2915 - categorical_accuracy: 0.0258
Epoch 2/20
3448/3448 [==============================] - 34s 10ms/step - loss: 7.0847 - categorical_accuracy: 0.0331
Epoch 3/20
3448/3448 [==============================] - 34s 10ms/step - loss: 6.8145 - 

## **Creating a dictionary to store the embeddings**

In [ ]:
weights = model.get_weights()[0]

embedding_dict = {}
for word in words: 
    embedding_dict.update({
        word: weights[unique_word_dict.get(word)]
        })
    
embedding_dict   

## **return similar words**

In [1]:
from sklearn.metrics.pairwise import euclidean_distances

distance_matrix = euclidean_distances(weights)


similar_words = {search_term: [id2word[idx] for idx in distance_matrix[word2id[search_term]-1].argsort()[1:15]+1] 
                   for search_term in ['گنج','خدا','دنیا']}

similar_words

NameError: ignored